In [16]:
import nltk
import pickle
from util.CustomedBleu.bleu import *
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction
from util.CustomedBleu.smooth_bleu import smooth_bleu
from util.CustomedBleu.bleu import _bleu

In [57]:
result = pickle.load(open('result.pkl','rb'))

In [58]:
preds = result['pred']
tgts = result['tgt']

In [17]:
def calculate_bleu(model, test_data_loader, summary_i2w, trgs, trimTilEOS):
    preds = compute_predictions(model, test_data_loader, summary_i2w, trimTilEOS)
    return calc_all_metrics(preds, trgs), preds


def compute_predictions(model, test_data_loader, summary_i2w, trimTilEOS):
    model.eval()

    preds = []
    for test_batch_data in test_data_loader:
        # trg = test_batch_data[-1]
        for i in range(len(test_batch_data)):
            test_batch_data[i] = move_to_device(test_batch_data[i])

        with torch.no_grad():
            if cf.modeltype == 'ast-att-gru':
                output = model(method_code=test_batch_data[0], method_sbt=test_batch_data[1],
                               method_summary=test_batch_data[2], use_teacher=False)[-1]
            elif cf.modeltype == 'att-gru' or cf.modeltype == 'codenn':
                output = model(method_code=test_batch_data[0], method_summary=test_batch_data[1], use_teacher=False)[-1]
            # elif cf.modeltype == "uml":
            #     output = model(
            else:
                raise Exception("Unrecognized Model: ", str(cf.modeltype))

            # output: batch_size, summary_length - 1, sum_vocab_size
            # pred: batch_size, summary_length - 1
            pred = torch.argmax(output, dim=2)

        # https://github.com/mcmillco/funcom/blob/41c737903a286e428493705e53c9c0e5946d2af6/bleu.py#L92
        # https://github.com/mcmillco/funcom/blob/41c737903a286e428493705e53c9c0e5946d2af6/bleu.py#L107
        # If p or t contain <s>, </s>, <UNK>, <NULL>,
        # they will be included as output.
        for i in range(cf.batch_size):
            p = pred[i].cpu().tolist()
            if trimTilEOS:
                if cf.EOS_token_id in p:  # truncate at first </s>
                    p = p[:p.index(cf.EOS_token_id) + 1]

            p = [summary_i2w[value] for value in p if
                    value not in [cf.PAD_token_id, cf.SOS_token_id, cf.EOS_token_id, cf.UNK_token_id]]

            preds.append(p)
            # t = trg[i].cpu().tolist()
            # t = [value for value in t if
            #      value not in [cf.PAD_token_id, cf.SOS_token_id, cf.EOS_token_id, cf.UNK_token_id]]
            # trgs.append([t])
    model.train()
    return preds


In [59]:
def bleu_so_far(refs, preds):
    # https://www.nltk.org/api/nltk.translate.html#nltk.translate.bleu_score.SmoothingFunction

    c_bleu1 = corpus_bleu(refs, preds, weights=(1, 0, 0, 0))
    c_bleu2 = corpus_bleu(refs, preds, weights=(0.5, 0.5, 0, 0))
    c_bleu3 = corpus_bleu(refs, preds, weights=(1/3, 1/3, 1/3, 0))
    c_bleu4 = corpus_bleu(refs, preds, weights=(0.25, 0.25, 0.25, 0.25))

    i_bleu2 = corpus_bleu(refs, preds, weights=(0, 1, 0, 0))
    i_bleu3 = corpus_bleu(refs, preds, weights=(0, 0, 1, 0))
    i_bleu4 = corpus_bleu(refs, preds, weights=(0, 0, 0, 1))

    c_bleu1 = round(c_bleu1 * 100, 2)
    c_bleu2 = round(c_bleu2 * 100, 2)
    c_bleu3 = round(c_bleu3 * 100, 2)
    c_bleu4 = round(c_bleu4 * 100, 2)
    i_bleu2 = round(i_bleu2 * 100, 2)
    i_bleu3 = round(i_bleu3 * 100, 2)
    i_bleu4 = round(i_bleu4 * 100, 2)

    ret = ''
    # https://machinelearningmastery.com/calculate-bleu-score-for-text-python/
    ret += ('Cumulative 4-gram BLEU (BLEU-4): %s\n' % c_bleu4)
    ret += 'Cumulative 1/2/3-gram BLEU: {}, {}, {}\n'.format(c_bleu1, c_bleu2, c_bleu3)
    ret += 'Individual 2/3/4-gram BLEU: {}, {}, {}\n'.format(i_bleu2, i_bleu3, i_bleu4)

    # Ba = corpus_bleu(refs, preds, smoothing_function=SmoothingFunction().method2,)
    # B1 = corpus_bleu(refs, preds, smoothing_function=SmoothingFunction().method2, weights=(1, 0, 0, 0))
    # B2 = corpus_bleu(refs, preds, smoothing_function=SmoothingFunction().method2, weights=(0, 1, 0, 0))
    # B3 = corpus_bleu(refs, preds, smoothing_function=SmoothingFunction().method2, weights=(0, 0, 1, 0))
    # B4 = corpus_bleu(refs, preds, smoothing_function=SmoothingFunction().method2, weights=(0, 0, 0, 1))
    #
    # Ba = round(Ba * 100, 2)
    # B1 = round(B1 * 100, 2)
    # B2 = round(B2 * 100, 2)
    # B3 = round(B3 * 100, 2)
    # B4 = round(B4 * 100, 2)
    #
    # ret = ('Smooth Bleu for %s functions\n' % (len(preds)))
    # ret += ('Ba %s\n' % Ba)
    # ret += ('B1 %s\n' % B1)
    # ret += ('B2 %s\n' % B2)
    # ret += ('B3 %s\n' % B3)
    # ret += ('B4 %s\n' % B4)

    sf = SmoothingFunction()
    all_score = 0.0
    count = 0
    for r, p in zip(refs, preds):
        # nltk bug: https://github.com/nltk/nltk/issues/2204
        if len(p) == 1:
            continue
        # i.e. sentence_bleu
        score = nltk.translate.bleu(r, p, smoothing_function=sf.method4)
        all_score += score
        count += 1
    emse_bleu = round(all_score/count * 100, 2)
    ret += 'EMSE BLEU: {}\n'.format(emse_bleu)

    r_str_list = []
    p_str_list = []
    for r, p in zip(refs, preds):
        if len(r[0]) == 0 or len(p) == 0:
            continue

        r_str_list.append([" ".join([str(token_id) for token_id in r[0]])])
        p_str_list.append(" ".join([str(token_id) for token_id in p]))

    bleu_list = smooth_bleu(r_str_list, p_str_list)
    ret += ('CodeBert Smooth Bleu: %f\n' % bleu_list[0])
    # for b in bleu_list:
    #     ret += ('%f\n' % b)

    bleu_score = _bleu(r_str_list, p_str_list)
    ret += ('Other Smooth Bleu: %f\n' % bleu_score)

    return ret, c_bleu4, bleu_list[0]

In [60]:
ret, c_bleu4, bleu_list = bleu_so_far(tgts, preds)

[[3395, 2570, 8, 2639, 552, 3415, 31, 284]] [405, 44, 8, 188, 78]
[[26110, 8, 170, 7947, 74, 31, 134, 415, 52, 3992]] [120, 47, 107, 111, 52, 8, 188]
[[2316, 47, 2269, 348, 52, 7947, 74, 154, 691, 2162]] [120, 47, 107, 111, 52, 8, 188]
[[2316, 8, 7947, 74, 684, 52, 7947, 154, 159, 47]] [120, 47, 107, 111, 52, 8, 188]
[[3977, 47, 2310, 52, 418, 690, 8, 143, 318, 364]] [120, 47, 107, 111, 52, 8, 188]
[[9302, 2605, 7947, 154, 159, 170, 7947, 74, 31, 47]] [120, 47, 107, 111, 52, 8]
[[120, 134, 7947, 15, 3415, 47, 684, 52, 1804, 1469]] [120, 47, 107, 111, 52, 8, 188]
[[120, 134, 7947, 15, 3415, 47, 684, 52, 2815, 1469]] [120, 47, 107, 111, 52, 8]
[[120, 47, 64, 15, 3415, 47, 1805, 78, 15, 502]] [120, 47, 107, 312, 52, 8, 188]
[[2514, 134, 7947, 74, 15, 224, 47, 9631, 500, 427]] [120, 47, 107, 111, 52, 8, 188]
[[120, 134, 7947, 15, 3415, 8, 1855, 52, 47, 25]] [224, 47, 107, 111, 52, 8, 188]
[[120, 134, 7947, 15, 3415, 8, 1855, 52, 47, 25]] [120, 47, 107, 211, 52, 8, 188]
[[120, 134, 7947, 15

[[120, 47, 9815, 15, 3415, 8, 1855, 52, 47, 25]] [120, 47, 107, 2693, 52, 8, 188]
[[120, 47, 9815, 15, 3415, 8, 1855, 52, 47, 25]] [224, 47, 107, 349, 52, 8]
[[120, 47, 9815, 15, 3415, 1793, 52, 635, 218, 3138]] [120, 47, 107, 170, 52, 8]
[[120, 47, 9815, 15, 3415, 1793, 52, 635, 218, 3138]] [120, 47, 107, 111, 52, 8, 188]
[[120, 47, 9815, 15, 3415, 1793, 52, 635, 218, 3138]] [120, 47, 107, 111, 52, 8, 188]
[[120, 47, 9815, 15, 3415, 1793, 52, 635, 218, 3138]] [120, 47, 107, 111, 52, 8]
[[120, 47, 9815, 15, 3415, 742, 3234, 1223, 635, 59]] [120, 47, 107, 111, 52, 8]
[[2316, 47, 2269, 348, 52, 7932, 154, 691, 2162, 21]] [120, 47, 107, 170, 52, 8]
[[120, 47, 9815, 15, 3415, 47, 318, 8, 451, 2362]] [120, 47, 107, 1112, 52, 8, 188]
[[2316, 8, 7932, 684, 52, 8006, 159, 47, 64, 684]] [224, 47, 107, 3284, 52, 8, 188]
[[2316, 134, 415, 52]] [224, 47, 107]
[[9302, 134, 1819, 52, 8006, 159, 170, 7932, 1776, 23]] [224, 47, 107, 170, 52, 8, 188]
[[46, 47, 9815, 72, 2973, 47, 7932, 580, 146, 7]] [1

[[224, 134, 427, 3383, 43, 15, 1987, 8, 25, 427]] [224, 47, 107, 111, 52, 8]
[[605, 47, 765, 367, 630, 427, 3383, 133]] [224, 47, 107, 111, 52, 8]
[[605, 47, 765, 629, 427, 3383, 133]] [224, 47, 107, 111, 52, 8]
[[605, 47, 765, 629, 427, 3383, 133]] [224, 47, 107, 111, 52, 8]
[[605, 47, 765, 11761, 427, 3383, 133]] [224, 47, 107, 111, 52, 8]
[[423, 47, 100, 1875, 26, 51, 666, 188, 100, 1875]] [394, 8, 188, 654]
[[8, 2370, 3372, 141, 2353, 59, 434, 435, 344, 544]] [394, 8, 188, 654]
[[81, 47, 100, 1875, 4236, 98, 8, 313, 309, 269]] [394, 8, 188, 313]
[[81, 47, 1846, 1875, 4236, 98, 8, 313, 309, 269]] [11, 8, 188, 313]
[[14188, 291, 27404, 7, 1882, 7, 47, 2365, 98, 8]] [668, 47, 107]
[[1278, 2931, 2881, 8, 240, 691, 15, 21, 1879, 1596]] [224, 47, 107, 111, 52, 8]
[[35, 8, 1848, 2931, 37, 7, 81, 8, 140, 1893]] [224, 47, 107]
[[41, 47, 2347, 240, 7, 47, 637, 4021, 8, 552]] [224, 47, 107, 251]
[[605, 8, 155, 2010, 26, 2179, 784]] [224, 47, 107, 654]
[[3387, 331, 291, 27404, 401, 729, 486, 8

[[227, 742, 415, 78, 7, 415, 4143, 3173, 10, 559]] [405, 44, 8, 188, 78, 141]
[[227, 458, 203, 7, 1449, 203]] [120, 8, 188, 55, 52, 8]
[[706, 48, 458, 203, 7, 8, 1845, 731, 5, 458]] [394, 8]
[[120, 2223, 569, 27, 1776, 552, 50, 2179]] [120, 8, 27, 52, 8]
[[120, 78, 52, 233, 63, 458]] [742, 2750]
[[706, 3591, 3822, 3591, 17, 18, 19, 219, 415, 379]] [120, 8, 415, 52, 8, 188, 371]
[[2856, 8, 14112, 249, 2030, 756, 7, 8, 1874, 841]] [394, 8]
[[120, 1780, 27, 552, 2280, 2316]] [120, 8, 27, 52, 8]
[[11]] [742, 2750]
[[8391]] [742, 2750]
[[2849, 8, 544, 381, 218, 39, 18, 88, 2146, 147]] [2762, 8]
[[2792, 47, 107]] [224, 47, 107]
[[513, 2030, 726, 26, 140, 792]] [394, 8, 78, 52, 8]
[[1]] [742, 2750]
[[1605, 282, 78, 141, 343, 59, 121, 11, 763, 218]] [120, 8, 188, 371]
[[3663, 664, 665, 72, 3504, 45, 345]] [394, 8, 188, 100]
[[706, 64, 3126, 224, 107, 3126, 111, 44, 147, 166]] [706, 8, 188, 371]
[[3138, 51, 8947]] [12, 8]
[[3138, 51, 623]] [394, 8, 841, 52, 8]
[[2374, 792, 2381, 52, 47, 2030, 1

[[221, 188, 500, 29, 27, 500, 141, 2231, 526, 50]] [120, 8, 78, 52, 8, 188, 500]
[[120, 174, 400, 332, 31, 460, 52, 415, 52, 400]] [120, 8]
[[405, 44, 240, 44, 2642, 1020, 50, 73, 218, 44]] [73, 8]
[[1350, 64, 427, 72, 67, 55, 2849, 55, 78]] [742, 2750]
[[120, 107, 80, 168, 111, 52, 637]] [120, 47, 107, 111, 52, 8, 188, 240]
[[509, 47, 251, 26, 8, 188, 446, 251, 50, 67]] [509, 8, 188, 249]
[[120, 249, 260, 618, 59, 541, 165, 7, 397, 270]] [120, 47, 107, 415, 52, 8]
[[11261, 8, 637, 7, 830, 8, 397, 270, 21, 34]] [120, 8, 188, 2217]
[[1959, 1780, 249, 400, 332, 59, 541, 7, 397, 270]] [120, 8, 654, 52, 8, 188]
[[2225, 655, 50, 7165, 116, 7, 2844, 270]] [120, 8, 188, 2217]
[[2374, 737, 270, 31, 249, 400, 120, 2217, 427, 44]] [120, 8, 188, 249]
[[2792, 47, 500, 52, 8, 25, 203, 26, 1216, 2329]] [120, 8, 671, 349, 52, 8]
[[2602, 64, 458, 31, 3355, 50, 31, 8, 233]] [742, 2750]
[[2602, 458, 31, 8, 3355, 72, 1871, 8, 174, 458]] [120, 8, 78, 52, 8, 188, 100]
[[1959, 400, 332, 159, 47, 654]] [120,

...

[[221, 29294, 460, 50, 719, 16632, 398]] [120, 47, 841, 52, 8]
[[221, 29294, 460, 13258, 16632, 398]] [120, 47, 841, 52, 8]
[[1591, 47, 467, 20, 8, 3610, 111, 69, 8, 36012]] [405, 44, 8, 188, 841, 52, 8]
[[221, 8, 460, 292, 72, 599, 100]] [224, 47, 107, 654]
[[221, 51, 3809, 439, 1, 3119, 444, 19059, 2291, 3190]] [224, 47, 107]
[[9226, 1350, 134, 168, 2291, 2874]] [224, 47, 107]
[[224, 47, 107, 2291, 2874]] [224, 47, 107]
[[2602, 134, 168, 2291, 2874]] [224, 47, 107]
[[221, 134, 168, 2291, 2874, 584, 2874]] [120, 47, 107]
[[221, 51, 3809, 439, 1, 3119, 444, 19059, 2291, 3190]] [120, 47, 107]
[[221, 51, 3809, 439, 1, 3119, 444, 19059, 2291, 3190]] [120, 47, 107, 787]
[[221, 51, 3809, 439, 1, 3119, 444, 19059, 2291, 3190]] [120, 47, 107]
[[221, 51, 2871, 584, 2291]] [224, 47, 107]
[[1350, 134, 168, 2291, 2874]] [224, 47, 107]
[[46, 47, 107, 460, 292, 29294, 8347]] [224, 47, 107, 654]
[[46, 47, 107, 2030, 821]] [120, 47, 107]
[[412, 47, 460, 292, 29294, 460, 8, 2030, 821, 8347]] [120, 47,

[[8405, 134, 427, 98, 7, 8, 541, 52, 8, 400]] [12, 8, 188, 427]
[[1591, 47, 64, 2874, 52, 21, 3203, 252, 253, 18438]] [742, 2750]
[[3003, 8, 2807, 72, 47, 3618, 52]] [12, 8, 188, 427]
[[668, 47, 147, 31, 404]] [394, 8, 188, 654]
[[14467, 1571, 8, 427, 213, 8, 541, 52, 8, 2807]] [1571, 8]
[[2141, 26, 787, 7, 7754, 47, 100, 290, 12992, 190]] [530, 8]
[[14467, 8, 427, 213, 8, 541, 52, 8, 400]] [1571, 8, 731, 427]
[[668, 47, 84, 2179, 404]] [224, 47, 107, 312]
[[2977, 8, 12876, 1708, 306]] [405, 8]
[[2082, 8, 427, 213, 8, 188, 416, 52, 8, 2807]] [120, 8, 416, 52, 8]
[[3395, 8, 268, 15592, 400, 182, 61, 47, 4350, 7402]] [120, 8, 348, 52, 8]
[[12, 8431, 1283]] [394, 8, 981, 52, 8]
[[108, 1283]] [742, 2750]
[[2255, 8, 137, 138, 145]] [73, 8]
[[2559, 21, 15592, 17581, 19842, 98, 370]] [742, 2750]
[[8421, 8, 8431, 213, 8, 541, 52, 8, 289, 400]] [405, 44, 8, 188, 854]
[[8421, 8, 3203, 213, 8, 541, 52, 21, 4407, 63]] [405, 8]
[[2072, 47, 17321, 59, 11014, 47, 7, 11014, 951]] [405, 44, 8, 188, 348

[[108, 21, 3190, 31, 3992, 714]] [742, 2750]
[[235, 655]] [742, 2750]
[[15624, 277, 277, 240]] [120, 8, 78, 52, 8]
[[15624, 277, 277, 240]] [120, 8, 277, 52, 8]
[[394, 8, 671, 635, 7, 8, 347, 10923, 4851, 5251]] [742, 2750]
[[8, 74, 52, 75, 26, 8351, 8, 1152, 2179, 75]] [394, 8]
[[1]] [120, 8, 981, 415, 52, 8, 188, 654]
[[8351, 8, 103, 75, 74]] [12, 8]
[[108, 21, 3190, 3101, 480, 16679]] [224, 47, 107]
[[1808, 134, 4720, 7, 61, 47, 75, 238]] [120, 8, 654, 52, 8, 188, 313]
[[27, 5374, 11841]] [542, 8, 188, 654]
[[27, 5375, 11841]] [27, 27, 27]
[[502, 331, 8, 188, 676, 141, 31, 8, 252, 11]] [405, 44, 8, 188, 654, 141]
[[9327, 2027, 452, 7, 8, 1908, 2027, 10, 80, 44]] [1959, 8, 188]
[[120, 832, 59, 1838]] [120, 8, 78, 52, 8]
[[1, 75, 1]] [12, 8]
[[317, 72]] [742, 2750]
[[3071, 134, 4720]] [3071, 8, 188, 313]
[[1, 75, 1]] [394, 8, 841, 52, 8]
[[142, 8, 637, 31, 47, 107, 370, 318, 47, 2362]] [224, 47, 107, 111, 52, 8]
[[31, 8, 460, 52, 329, 8351, 2016, 398]] [591, 8]
[[8351, 733, 398, 31, 3

[[1571, 2193, 1471, 865, 59, 803, 358, 52, 21, 654]] [120, 8, 654, 52, 8, 654]
[[11, 8, 79, 7, 8, 174, 2193, 690]] [394, 8]
[[2374, 8, 1845, 731, 416, 52, 47, 1576, 59, 8]] [120, 8, 348, 52, 8, 188, 654]
[[11, 8, 79, 7, 8, 174, 2193, 690]] [394, 8]
[[2374, 143, 416, 52, 47, 1576, 31, 8, 188, 74]] [120, 8, 203, 52, 8, 188, 654]
[[2374, 731, 416, 52, 47, 1576, 31, 8, 188, 74]] [120, 8, 348, 52, 8]
[[2374, 731, 416, 52, 47, 1576, 31, 8, 188, 74]] [120, 8, 348, 52, 8, 654]
[[120, 232, 8, 1900, 31, 654, 80, 44, 8, 654]] [120, 8, 348, 52, 8, 188, 654]
[[120, 232, 8, 1900, 31, 654, 80, 44, 8, 654]] [405, 44, 8, 188, 654]
[[120, 232, 8, 1900, 31, 654, 80, 44, 8, 654]] [120, 8, 348, 52, 8, 188, 654]
[[3219, 47, 654, 3219, 1583, 7, 460, 47, 107, 654]] [120, 47, 654, 52, 8, 188, 654]
[[2699, 51, 3460, 59, 47, 654, 29, 30, 72, 676]] [120, 8, 654, 52, 8, 188, 654]
[[1, 1]] [120, 47, 654, 52, 8]
[[200, 8, 111, 2163, 9013, 427]] [405, 8]
[[200, 8, 111, 2163, 251, 1767, 427]] [405, 8]
[[200, 8, 111, 2

[[2311, 8, 25, 854, 69, 8, 2053, 50, 2602, 8]] [405, 8, 854]
[[21, 141, 8, 3553, 615, 4457, 213, 364, 535, 232]] [12, 47]
[[2269, 2327, 2259]] [12, 47, 107]
[[3395, 47, 3796, 59, 8, 167, 80, 281, 774, 7]] [394, 8]
[[2046, 2046]] [12, 8]
[[27, 654]] [12, 8, 188, 500]
[[3163, 47, 3163]] [120, 8, 654, 52, 8, 188, 654]
[[2602, 8, 276, 1451, 7717, 26, 47, 1451, 52, 8]] [394, 8]
[[2602, 8, 276, 1449, 1470, 26, 47, 1449, 52, 8]] [394, 8, 78, 52, 8]
[[2559, 47, 2031, 100, 23925, 1866, 8, 3560, 7, 18]] [405, 8, 188, 654]
[[3949, 8, 3938, 2320, 98, 67, 693, 2175, 145]] [742, 2750]
[[120, 47, 107, 854, 97, 98, 47, 2442, 52, 21]] [224, 47, 107]
[[3395, 47, 3796, 59, 8, 451, 774, 7, 8, 167]] [742, 2750]
[[3395, 47, 3796, 59, 8, 167, 774, 7, 8, 281]] [742, 2750]
[[8922, 18438, 55, 22032, 4720, 1815, 438, 9832, 108, 5074]] [12, 8]
[[9248, 4132, 654]] [120, 8]
[[2311, 693, 2140, 853, 8, 2320, 2204, 141, 9209, 98]] [405, 44, 8, 188, 854]
[[9248, 8749]] [224, 47, 107]
[[8922]] [120, 8, 654]
[[8922, 108,

In [61]:
print(ret)

Cumulative 4-gram BLEU (BLEU-4): 3.32
Cumulative 1/2/3-gram BLEU: 11.45, 6.83, 4.48
Individual 2/3/4-gram BLEU: 4.07, 1.93, 1.36
EMSE BLEU: 7.77
CodeBert Smooth Bleu: 11.530597
Other Smooth Bleu: 10.420000



In [32]:
t1 = [[[1,0,5,6]],[[2,4,3,7]]]
p1 = [[1,0,5],[2,4,3]]
r1 = bleu_so_far(t1, p1)
print(r1[0])

Cumulative 4-gram BLEU (BLEU-4): 0.0
Cumulative 1/2/3-gram BLEU: 71.65, 71.65, 71.65
Individual 2/3/4-gram BLEU: 71.65, 71.65, 0.0
EMSE BLEU: 43.22
CodeBert Smooth Bleu: 77.880078
Other Smooth Bleu: 67.030000



/home/yanlwang/anaconda3/lib/python3.7/site-packages/nltk/translate/bleu_score.py:523: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


In [56]:
t2 = [['I have a dream'], ['Go to the school']]
p2 = ['I have a', 'Go to the']
r2 = bleu_so_far(t2, p2)
print(r2[0])

['I have a dream'] I have a
['Go to the school'] Go to the
0.9317923767769414
2
Cumulative 4-gram BLEU (BLEU-4): 46.55
Cumulative 1/2/3-gram BLEU: 46.55, 46.55, 46.55
Individual 2/3/4-gram BLEU: 46.55, 46.55, 46.55
EMSE BLEU: 46.59
CodeBert Smooth Bleu: 48.095411
Other Smooth Bleu: 44.370000



In [50]:
sf = SmoothingFunction()
nltk.translate.bleu([['I have a dream']], ['I have a'], smoothing_function=sf.method4)

0

In [52]:
tgts[0]

['makes sure the fast path emits in order\n']

In [54]:
preds[0]

'checks if the given value\n'

In [45]:
len(t2)

2